In [ ]:
import geopandas as gpd
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import matplotlib.font_manager
import matplotlib

# Read the shapefiles
shapefile1 = gpd.read_file('DIRECTORY_PATH/inpe_file_2008_2020_valid_newCRS_ClassificationSuitability.shp')
shapefile2 = gpd.read_file('DIRECTORY_PATH/inpe_file_2008_2020_valid_legal&illegalclassification_ClassificationSuitability.shp')

# Convert the 'year' column in shapefile1 to integers
shapefile1['year'] = shapefile1['year'].astype(int)

# Convert the 'year' column in shapefile2 to integers
shapefile2['year'] = shapefile2['year'].astype(int)

# Filter shapefile1 for soy suitable deforestation
soy_suitable_deforestation = shapefile1[(shapefile1['pixel_1'] > shapefile1['pixel_0']) | (shapefile1['pixel_2'] > shapefile1['pixel_0'])]

# Filter shapefile2 for legal soy suitable deforestation
legal_soy_suitable_deforestation = shapefile2[(shapefile2['legality'] == 'legal') & ((shapefile2['pixel_1'] > shapefile2['pixel_0']) | (shapefile2['pixel_2'] > shapefile2['pixel_0']))]

# Group and sum the deforestation for each year using the combined range of years
total_deforestation = shapefile1.groupby('year')['area_km'].sum()
soy_suitable_deforestation_sum = soy_suitable_deforestation.groupby('year')['area_km'].sum()

# Group and sum legal soy suitable deforestation by year based on shapefile2
legal_soy_suitable_deforestation_sum = legal_soy_suitable_deforestation.groupby('year')['area'].sum()

# Combine the years from both shapefiles and get unique values
all_years = pd.concat([shapefile1['year'], shapefile2['year']]).astype(int).unique()

# Create a new DataFrame with years as the index
new_df = pd.DataFrame(index=all_years)

# Reindex the total deforestation, soy suitable deforestation, and legal soy suitable deforestation
total_deforestation_reindexed = total_deforestation.reindex(all_years, fill_value=0)
soy_suitable_deforestation_sum_reindexed = soy_suitable_deforestation_sum.reindex(all_years, fill_value=0)
legal_soy_suitable_deforestation_sum_reindexed = legal_soy_suitable_deforestation_sum.reindex(all_years, fill_value=0)

# Calculate other deforestation (difference between total deforestation and soy suitable deforestation)
other_deforestation = total_deforestation_reindexed - soy_suitable_deforestation_sum_reindexed

# Calculate other soy suitable deforestation (difference between soy suitable deforestation and legal soy suitable deforestation)
other_soy_suitable_deforestation = soy_suitable_deforestation_sum_reindexed - legal_soy_suitable_deforestation_sum_reindexed

# Add columns for each type of deforestation
new_df['Other Deforestation'] = other_deforestation.values
new_df['Illegal Soy Suitable Deforestation'] = other_soy_suitable_deforestation.values
new_df['Legal Soy Suitable Deforestation'] = legal_soy_suitable_deforestation_sum_reindexed.values

# Define the years and categories
years = new_df.index
other_deforestation = new_df['Other Deforestation']
other_soy_suitable_deforestation = new_df['Illegal Soy Suitable Deforestation']
legal_soy_suitable_deforestation = new_df['Legal Soy Suitable Deforestation']

# Plot the stacked bar chart
plt.figure(figsize=(10, 6))

# Plot other deforestation (bottom part of the bars) in gray
plt.bar(years, other_deforestation, color='#969696', alpha=1, label='Other Deforestation')

# Plot other soy suitable deforestation (middle part of the bars)
plt.bar(years, other_soy_suitable_deforestation, bottom=other_deforestation, color='#fec44f', alpha=1, label='Illegal Soy Suitable Deforestation')

# Plot legal soy suitable deforestation (top part of the bars)
plt.bar(years, legal_soy_suitable_deforestation, bottom=other_deforestation + other_soy_suitable_deforestation, color='#cc4c02', alpha=1, label='Legal Soy Suitable Deforestation')

# Add labels and legend
plt.xlabel('')
plt.ylabel('Total Deforestation (km²)', fontsize=14, weight='bold')
plt.title('')

# Create custom legend
legal_soy_patch = mpatches.Patch(color='#cc4c02', label='Legal deforestation on soy-suitable land')
other_soy_patch = mpatches.Patch(color='#fec44f', label='Illegal deforestation on soy-suitable land')
other_patch = mpatches.Patch(color='#969696', label='Deforestation on other types of land')

plt.legend(handles=[legal_soy_patch, other_soy_patch, other_patch], loc='upper center')

# Show plot
plt.xticks(years, fontsize=11, fontweight='bold')
plt.tight_layout()
plt.show()